In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

In [24]:
df = pd.read_csv('C:/Users/KIHyuk/Desktop/dacon_data/Kaggle_영화데이터/tmdb_5000_movies.csv')

In [25]:
df.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [26]:
df = df[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]

In [27]:
from ast import literal_eval

df['genres'] = df['genres'].apply(literal_eval)
df['keywords'] = df['keywords'].apply(literal_eval)

In [28]:
df['genres'] = df['genres'].apply(lambda x : [y['name'] for y in x])
df['keywords'] = df['keywords'].apply(lambda x : [y['name'] for y in x])
df[['genres','keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon..."


In [13]:
from sklearn.feature_extraction.text import CountVectorizer

df['genres_literal'] = df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(df['genres_literal'])
print(genre_mat.shape)

from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)

genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind[:1])

title_movie = df[df['title'] == 'The Godfather']

title_index = title_movie.index.values

genre_sim_sorted_ind[title_index,:10]

(4803, 276)


In [65]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df['genres_literal'] = df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(df['genres_literal'])

In [79]:
title_movie = df[df['title'] == 'The Godfather']['genres_literal']
# title_movie
target_mat = count_vect.transform(pd.Series("Drama Crime"))

In [80]:
genre_sim = cosine_similarity(target_mat, genre_mat)

In [81]:
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]
top10_sim = genre_sim_sorted_ind[:1,:10]
top10_sim = top10_sim.reshape(-1)

In [82]:
df.iloc[top10_sim]

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
2731,240,The Godfather: Part II,"[Drama, Crime]",8.3,3338,105.792936,"[italo-american, cuba, vororte, melancholy, pr...",In the continuing saga of the Corleone crime f...,Drama Crime
1243,203,Mean Streets,"[Drama, Crime]",7.2,345,17.002096,"[epilepsy, protection money, secret love, mone...","A small-time hood must choose from among love,...",Drama Crime
3636,36351,Light Sleeper,"[Drama, Crime]",5.7,15,6.063868,"[suicide, drug dealer, redemption, addict, exi...",A drug dealer with upscale clientele is having...,Drama Crime
1946,11699,The Bad Lieutenant: Port of Call - New Orleans,"[Drama, Crime]",6.0,326,17.339852,"[police brutality, organized crime, policeman,...","Terrence McDonagh, a New Orleans Police sergea...",Drama Crime
2640,400,Things to Do in Denver When You're Dead,"[Drama, Crime]",6.7,85,6.932221,"[father son relationship, bounty hunter, boat,...",A mafia film in Tarantino style with a star-st...,Drama Crime
4065,364083,Mi America,"[Drama, Crime]",0.0,0,0.039007,"[new york state, hate crime]",A hate-crime has been committed in a the small...,Drama Crime
1847,769,GoodFellas,"[Drama, Crime]",8.2,3128,63.654244,"[prison, based on novel, florida, 1970s, mass ...","The true story of Henry Hill, a half-Irish, ha...",Drama Crime
4217,9344,Kids,"[Drama, Crime]",6.8,279,13.291991,"[puberty, first time]",A controversial portrayal of teens in New York...,Drama Crime
883,640,Catch Me If You Can,"[Drama, Crime]",7.7,3795,73.944049,"[con man, biography, fbi agent, overhead camer...","A true story about Frank Abagnale Jr. who, bef...",Drama Crime
3866,598,City of God,"[Drama, Crime]",8.1,1814,44.356711,"[male nudity, street gang, brazilian, photogra...",Cidade de Deus is a shantytown that started du...,Drama Crime
